## Annihilation operator for a single boson

The cutoff L needs to be a power of 2 if we want to use qubit operators. Otherwise `qiskit` will make everything a single qubit operator.

It is convenient to use sparse matrices if we want to save memory. However, `qiskit` will transform them to dense matrices when using objects like `MatrixOp`

In [1]:
import numpy as np
from scipy.sparse import diags
L = 2  # cutoff for Fock space
a = diags(np.sqrt(np.linspace(1,L-1,L-1)),offsets=1)

### Representation in QISKIT

A qubit operator in `qiskit` can be initialized from a matrix using `MatrixOp`

In [2]:
from qiskit.opflow import MatrixOp

qubitOp = MatrixOp(primitive=a)
print(qubitOp.num_qubits)

1


Express the annihilation operator of a single boson in terms of N-qubit operators represented by Pauli matrices (2x2)

In [3]:
a_pauli = qubitOp.to_pauli_op()
print(a_pauli)

SummedOp([
  0.5 * X,
  0.5j * Y
])


In [4]:
a_pauli.to_matrix()

array([[0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j]])

In [5]:
qubitOp.to_matrix()

array([[0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j]])

The Pauli representation and the matrix representation match.

## Identity operator for a single boson

In [6]:
from scipy.sparse import identity
iden = identity(L)

In [7]:
qubitOp = MatrixOp(primitive=iden)
print(qubitOp.num_qubits)

1


In [8]:
i_pauli = qubitOp.to_pauli_op()
print(i_pauli)

I


In [9]:
i_pauli.to_matrix()

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [10]:
qubitOp.to_matrix()

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

## Use the class `Operator`

The `Operator` class likes `numpy` arrays or lists.

In [11]:
from qiskit.quantum_info.operators import Operator

a_op = Operator(a.toarray())
print(a_op)

Operator([[0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j]],
         input_dims=(2,), output_dims=(2,))


In [12]:
i_op = Operator(iden.toarray())
print(i_op)

Operator([[1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j]],
         input_dims=(2,), output_dims=(2,))


The tensor product is:

In [13]:
a_op^i_op

Operator([[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))

and it keeps track of the dimensions of the individual sub-systems.

# Create the Hamiltonian for 6 interacting bosons

Try to use the information in [this QISKIT tutorial](https://qiskit.org/documentation/tutorials/operators/01_operator_flow.html) and [this other one](https://qiskit.org/documentation/tutorials/circuits_advanced/02_operators_overview.html)

In [14]:
Nmat = 6  # BMN2 for SU(2) has 6 bosonic matrices

In [15]:
from qiskit.opflow.list_ops import ListOp, TensoredOp

boson = ListOp([i_pauli]*6)
print(boson)

ListOp([
  I,
  I,
  I,
  I,
  I,
  I
])


In [16]:
print(boson[0])

I


Since assignment is not possible, we construct the Nmat operators manually

In [17]:
# this does not work because the dimension of the spaces is not consisten
# boson0 = a_pauli^(i_pauli^5)

In [18]:
boson0 = TensoredOp([a_pauli, i_pauli, i_pauli, i_pauli, i_pauli, i_pauli])

In [19]:
print(boson0)

TensoredOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


In [20]:
boson0.num_qubits

6

In [21]:
boson0.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 1 stored elements in Compressed Sparse Row format>

In [22]:
boson0.to_matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])

In [23]:
boson0.to_matrix().shape

(64, 64)

In [24]:
boson0.primitive_strings()

{'Pauli'}

In [25]:
boson1 = TensoredOp([i_pauli, a_pauli, i_pauli, i_pauli, i_pauli, i_pauli])

In [26]:
print(boson1)

TensoredOp([
  I,
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I
])


In [27]:
boson1.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 1 stored elements in Compressed Sparse Row format>

In [28]:
sparse = boson1.to_spmatrix()
type(sparse)

scipy.sparse.csr.csr_matrix

In [29]:
sparse.shape

(2, 2)

In [30]:
boson1.to_matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])

In [31]:
full = boson1.to_matrix()
len(full)

64

In [32]:
type(full)

numpy.ndarray

In [33]:
full.shape

(64, 64)

We can use a list of `ListOp` and then perform the tensor product using `TensoredOp`

In [34]:
# generically speaking, we construct the list of bosons and then take the outer product
a_list = []  # this will contain a1...a6 as a list of ListOp
for i in np.arange(0,Nmat):  # loop over all operators
    operator_list = [i_pauli] * Nmat  # only the identity repeated Nmat times
    operator_list[i] = a_pauli  # the i^th element is now the annihilation operator for a single boson
    a_list.append(ListOp(operator_list))


In [35]:
print(a_list[0])

ListOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


This is simply a list of 6 operators in Pauli form.

In [36]:
a_list[0].primitive_strings()

{'Pauli'}

In [37]:
# here we create a list of operators which are the tensor products...
a_tensor = [TensoredOp(a) for a in a_list]


In [38]:
print(a_tensor[0])

TensoredOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


In [39]:
assert a_tensor[0] == boson0

The tensor operator create from the list of Pauli operators is the same as the one created by manually doing the tensor product.

In [40]:
a_tensor[0].primitive_strings()

{'Pauli'}

In [41]:
print(a_tensor[0].to_pauli_op())

0.5 * XIIIII
+ 0.5j * YIIIII


The representation is already in terms of Pauli operators.

In [42]:
a_tensor[0].num_qubits

6

In [43]:
sparse = a_tensor[0].to_spmatrix()
type(sparse)

scipy.sparse.csr.csr_matrix

In [44]:
sparse.shape

(2, 2)

In [45]:
full = a_tensor[0].to_matrix()
len(full)

64

In [46]:
full.shape

(64, 64)

The matrix operator can be created from this matrix for example

In [47]:
boson0_op = Operator(full)

In [48]:
boson0_op

Operator([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          ...,
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))

In [49]:
testop = Operator(a_tensor[0])
print(testop)

Operator([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          ...,
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


From the test above we see that we can use the Pauli representation to get a full matrix operator directly.

We also create the identity in this new tensor space of 6 bosons

In [50]:
i_tensor = TensoredOp(ListOp([i_pauli] * Nmat))

In [51]:
print(i_tensor)

TensoredOp([
  I,
  I,
  I,
  I,
  I,
  I
])


In [52]:
i_tensor.num_qubits

6

In [53]:
full = i_tensor.to_matrix()
len(full)

64

In [54]:
print(i_tensor.to_spmatrix())

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)


In [55]:
i_op = Operator(i_tensor)
print(i_op)

Operator([[1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          ...,
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 1.+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


## Create the position operators

In [96]:
# for each boson they are constructed using a and adag
x_tensor = [1/np.sqrt(2)*(~a + a) for a in a_tensor]

In [97]:
print(x_tensor[0])

0.7071067811865475 * SummedOp([
  TensoredOp([
    SummedOp([
      0.5 * X,
      -0.5j * Y
    ]),
    I,
    I,
    I,
    I,
    I
  ]),
  TensoredOp([
    SummedOp([
      0.5 * X,
      0.5j * Y
    ]),
    I,
    I,
    I,
    I,
    I
  ])
])


In [98]:
print(a_tensor[0],~a_tensor[0])

TensoredOp([
  SummedOp([
    0.5 * X,
    0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
]) TensoredOp([
  SummedOp([
    0.5 * X,
    -0.5j * Y
  ]),
  I,
  I,
  I,
  I,
  I
])


In [99]:
x0_op = Operator(x_tensor[0])
print(x0_op)

Operator([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          ...,
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


## Create the full Hamiltonian

In [100]:
from qiskit.opflow.list_ops import SummedOp

In [101]:
H_zero = 0.5*Nmat*i_tensor

In [102]:
print(H_zero)

3.0 * TensoredOp([
  I,
  I,
  I,
  I,
  I,
  I
])


In [103]:
H_zero.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

In [104]:
### Harmonic oscillator
# this should be summed over all the bosons (Nmat)
H_list = [H_zero]
for op in a_tensor:
    H_list.append((~op @ op))
H_osc = SummedOp(H_list)


In [105]:
print(H_osc)

SummedOp([
  3.0 * TensoredOp([
    I,
    I,
    I,
    I,
    I,
    I
  ]),
  ComposedOp([
    TensoredOp([
      SummedOp([
        0.5 * X,
        -0.5j * Y
      ]),
      I,
      I,
      I,
      I,
      I
    ]),
    TensoredOp([
      SummedOp([
        0.5 * X,
        0.5j * Y
      ]),
      I,
      I,
      I,
      I,
      I
    ])
  ]),
  ComposedOp([
    TensoredOp([
      I,
      SummedOp([
        0.5 * X,
        -0.5j * Y
      ]),
      I,
      I,
      I,
      I
    ]),
    TensoredOp([
      I,
      SummedOp([
        0.5 * X,
        0.5j * Y
      ]),
      I,
      I,
      I,
      I
    ])
  ]),
  ComposedOp([
    TensoredOp([
      I,
      I,
      SummedOp([
        0.5 * X,
        -0.5j * Y
      ]),
      I,
      I,
      I
    ]),
    TensoredOp([
      I,
      I,
      SummedOp([
        0.5 * X,
        0.5j * Y
      ]),
      I,
      I,
      I
    ])
  ]),
  ComposedOp([
    TensoredOp([
      I,
      I,
      I,
      SummedOp([
  

In [106]:
type(H_osc)

qiskit.opflow.list_ops.summed_op.SummedOp

In [107]:
H_osc.primitive_strings()

{'Pauli'}

The entire Pauli string representation can be obtained with

In [108]:
H_osc.to_pauli_op()

SummedOp([PauliOp(Pauli('IIIIII'), coeff=6.0), PauliOp(Pauli('ZIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIII'), coeff=-0.5), PauliOp(Pauli('IIZIII'), coeff=-0.5), PauliOp(Pauli('IIIZII'), coeff=-0.5), PauliOp(Pauli('IIIIZI'), coeff=-0.5), PauliOp(Pauli('IIIIIZ'), coeff=-0.5)], coeff=1.0, abelian=False)

It looks like a sum of 7 different Pauli terms is what makes a simple harmonic oscillator

In [109]:
H_osc.num_qubits

6

In [70]:
H_osc.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

We can write it as a matrix operator using the same expressions we used before for the single operators:

In [71]:
H0_op = Operator(H_osc)
print(H0_op)

Operator([[729.+0.j,   0.+0.j,   0.+0.j, ...,   0.+0.j,   0.+0.j,
             0.+0.j],
          [  0.+0.j, 730.+0.j,   0.+0.j, ...,   0.+0.j,   0.+0.j,
             0.+0.j],
          [  0.+0.j,   0.+0.j, 730.+0.j, ...,   0.+0.j,   0.+0.j,
             0.+0.j],
          ...,
          [  0.+0.j,   0.+0.j,   0.+0.j, ..., 734.+0.j,   0.+0.j,
             0.+0.j],
          [  0.+0.j,   0.+0.j,   0.+0.j, ...,   0.+0.j, 734.+0.j,
             0.+0.j],
          [  0.+0.j,   0.+0.j,   0.+0.j, ...,   0.+0.j,   0.+0.j,
           735.+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


### Test `SparsePauliOp`

We can try this [object](https://qiskit.org/documentation/stubs/qiskit.quantum_info.SparsePauliOp.html#qiskit.quantum_info.SparsePauliOp.from_list)

In [72]:
from qiskit.quantum_info import SparsePauliOp

In [73]:
H0_sp = SparsePauliOp.from_operator(H0_op)
print(H0_sp)

SparsePauliOp([[False, False, False, False, False, False, False, False,
                False, False, False, False],
               [False, False, False, False, False, False,  True, False,
                False, False, False, False],
               [False, False, False, False, False, False, False,  True,
                False, False, False, False],
               [False, False, False, False, False, False, False, False,
                 True, False, False, False],
               [False, False, False, False, False, False, False, False,
                False,  True, False, False],
               [False, False, False, False, False, False, False, False,
                False, False,  True, False],
               [False, False, False, False, False, False, False, False,
                False, False, False,  True]],
              coeffs=[ 7.32e+02+0.j, -5.00e-01+0.j, -5.00e-01+0.j, -5.00e-01+0.j, -5.00e-01+0.j,
 -5.00e-01+0.j, -5.00e-01+0.j])


It is saved as a table of Pauli matrices

In [74]:
H0_sp.table

PauliTable([[False,False,False,False,False,False,False,False,False,False,
             False,False],
            [False,False,False,False,False,False, True,False,False,False,
             False,False],
            [False,False,False,False,False,False,False, True,False,False,
             False,False],
            [False,False,False,False,False,False,False,False, True,False,
             False,False],
            [False,False,False,False,False,False,False,False,False, True,
             False,False],
            [False,False,False,False,False,False,False,False,False,False,
              True,False],
            [False,False,False,False,False,False,False,False,False,False,
             False, True]])

In [75]:
H0_sp.num_qubits

6

In [76]:
H0_sp.dim

(64, 64)

In [77]:
H0_sp.input_dims()

(2, 2, 2, 2, 2, 2)

Can it be created from a `SummedOp` object?

In [78]:
H0_sp = SparsePauliOp.from_operator(H_osc)
print(H0_sp)

SparsePauliOp([[False, False, False, False, False, False, False, False,
                False, False, False, False],
               [False, False, False, False, False, False,  True, False,
                False, False, False, False],
               [False, False, False, False, False, False, False,  True,
                False, False, False, False],
               [False, False, False, False, False, False, False, False,
                 True, False, False, False],
               [False, False, False, False, False, False, False, False,
                False,  True, False, False],
               [False, False, False, False, False, False, False, False,
                False, False,  True, False],
               [False, False, False, False, False, False, False, False,
                False, False, False,  True]],
              coeffs=[ 7.32e+02+0.j, -5.00e-01+0.j, -5.00e-01+0.j, -5.00e-01+0.j, -5.00e-01+0.j,
 -5.00e-01+0.j, -5.00e-01+0.j])


Yes, because they are basically equivalent.

### Find minimum (classical) eigenvalue

In [79]:
from qiskit.algorithms import NumPyMinimumEigensolver

mes = NumPyMinimumEigensolver()

# we need to use the Pauli representation and not the Operator representation
result = mes.compute_minimum_eigenvalue(H_osc)
print(result.eigenvalue)

(3+0j)


## Interaction terms: quartic potential

In [110]:
quartic1 = SummedOp([x_tensor[2]@x_tensor[2]@x_tensor[3]@x_tensor[3], \
                    x_tensor[2]@x_tensor[2]@x_tensor[4]@x_tensor[4], \
                    x_tensor[1]@x_tensor[1]@x_tensor[3]@x_tensor[3], \
                    x_tensor[1]@x_tensor[1]@x_tensor[5]@x_tensor[5], \
                    x_tensor[0]@x_tensor[0]@x_tensor[4]@x_tensor[4], \
                    x_tensor[0]@x_tensor[0]@x_tensor[5]@x_tensor[5]])
                    

In [111]:
quartic2 = SummedOp([x_tensor[0]@x_tensor[2]@x_tensor[3]@x_tensor[5],\
                    x_tensor[0]@x_tensor[1]@x_tensor[3]@x_tensor[4],\
                    x_tensor[1]@x_tensor[2]@x_tensor[4]@x_tensor[5]], coeff = -2.0)

In [82]:
print(quartic2)

-2.0 * SummedOp([
  ComposedOp([
    0.7071067811865475 * SummedOp([
      TensoredOp([
        SummedOp([
          0.5 * X,
          -0.5j * Y
        ]),
        I,
        I,
        I,
        I,
        I
      ]),
      -1.0 * TensoredOp([
        SummedOp([
          0.5 * X,
          0.5j * Y
        ]),
        I,
        I,
        I,
        I,
        I
      ])
    ]),
    0.7071067811865475 * SummedOp([
      TensoredOp([
        I,
        I,
        SummedOp([
          0.5 * X,
          -0.5j * Y
        ]),
        I,
        I,
        I
      ]),
      -1.0 * TensoredOp([
        I,
        I,
        SummedOp([
          0.5 * X,
          0.5j * Y
        ]),
        I,
        I,
        I
      ])
    ]),
    0.7071067811865475 * SummedOp([
      TensoredOp([
        I,
        I,
        I,
        SummedOp([
          0.5 * X,
          -0.5j * Y
        ]),
        I,
        I
      ]),
      -1.0 * TensoredOp([
        I,
        I,
        I,
        S

In [83]:
type(quartic2)

qiskit.opflow.list_ops.summed_op.SummedOp

In [84]:
quartic2.primitive_strings()

{'Pauli'}

The full potential is the sum of those two terms:

In [112]:
### Quartic Interaction
V =  quartic1 + quartic2

In [113]:
V.num_qubits

6

In [114]:
V.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

We can make an operator with this

In [88]:
V_op = Operator(V)
print(V_op)

Operator([[1.5+0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
          [0. +0.j, 1.5+0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
          [0. +0.j, 0. +0.j, 1.5+0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
          ...,
          [0. +0.j, 0. +0.j, 0. +0.j, ..., 1.5+0.j, 0. +0.j, 0. +0.j],
          [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 1.5+0.j, 0. +0.j],
          [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 1.5+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


## Define the 'tHooft coupling and the Full Hamiltonian

In [115]:
g2 = 0.2

In [116]:
### Full Hamiltonian
H = H_osc + 0.5*g2*V

In [91]:
type(H)

qiskit.opflow.list_ops.summed_op.SummedOp

In [117]:
H.to_pauli_op()

SummedOp([PauliOp(Pauli('IIIIII'), coeff=6.150000000000002), PauliOp(Pauli('ZIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIII'), coeff=-0.5), PauliOp(Pauli('IIZIII'), coeff=-0.5), PauliOp(Pauli('IIIZII'), coeff=-0.5), PauliOp(Pauli('IIIIZI'), coeff=-0.5), PauliOp(Pauli('IIIIIZ'), coeff=-0.5), PauliOp(Pauli('XXIXXI'), coeff=-0.04999999999999999), PauliOp(Pauli('XIXXIX'), coeff=-0.04999999999999999), PauliOp(Pauli('IXXIXX'), coeff=-0.04999999999999999)], coeff=1.0, abelian=False)

In [93]:
H.num_qubits

6

# Diagonalize the Hamiltonian to find the groundstate

In [118]:
from qiskit.algorithms import NumPyMinimumEigensolver

mes = NumPyMinimumEigensolver()

result = mes.compute_minimum_eigenvalue(H)
print(result.eigenvalue)

(3+0j)


In [95]:
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.00000


I can not get anything different from 3.000 even if I have the interactions turned on...

In [119]:
H_op = Operator(H)
print(H_op)

Operator([[729.00015+0.j,   0.     +0.j,   0.     +0.j, ...,
             0.     +0.j,   0.     +0.j,   0.     +0.j],
          [  0.     +0.j, 730.00015+0.j,   0.     +0.j, ...,
             0.     +0.j,   0.     +0.j,   0.     +0.j],
          [  0.     +0.j,   0.     +0.j, 730.00015+0.j, ...,
             0.     +0.j,   0.     +0.j,   0.     +0.j],
          ...,
          [  0.     +0.j,   0.     +0.j,   0.     +0.j, ...,
           734.00015+0.j,   0.     +0.j,   0.     +0.j],
          [  0.     +0.j,   0.     +0.j,   0.     +0.j, ...,
             0.     +0.j, 734.00015+0.j,   0.     +0.j],
          [  0.     +0.j,   0.     +0.j,   0.     +0.j, ...,
             0.     +0.j,   0.     +0.j, 735.00015+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


In [122]:
type(H_op)

qiskit.quantum_info.operators.operator.Operator

In [120]:
H_qop = MatrixOp(H_op)
print(H_qop)

Operator([[729.00015+0.j,   0.     +0.j,   0.     +0.j, ...,
             0.     +0.j,   0.     +0.j,   0.     +0.j],
          [  0.     +0.j, 730.00015+0.j,   0.     +0.j, ...,
             0.     +0.j,   0.     +0.j,   0.     +0.j],
          [  0.     +0.j,   0.     +0.j, 730.00015+0.j, ...,
             0.     +0.j,   0.     +0.j,   0.     +0.j],
          ...,
          [  0.     +0.j,   0.     +0.j,   0.     +0.j, ...,
           734.00015+0.j,   0.     +0.j,   0.     +0.j],
          [  0.     +0.j,   0.     +0.j,   0.     +0.j, ...,
             0.     +0.j, 734.00015+0.j,   0.     +0.j],
          [  0.     +0.j,   0.     +0.j,   0.     +0.j, ...,
             0.     +0.j,   0.     +0.j, 735.00015+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


In [121]:
type(H_qop)

qiskit.opflow.primitive_ops.matrix_op.MatrixOp

In [124]:
result = mes.compute_minimum_eigenvalue(H_qop)
print(result.eigenvalue)

(729.0001498800242-2.8362595049852713e-16j)


In [126]:
result = mes.compute_minimum_eigenvalue(H_osc)
print(result.eigenvalue)

(3+0j)


In [128]:
H0_qop = MatrixOp(Operator(H_osc))
print(H0_qop)

Operator([[729.+0.j,   0.+0.j,   0.+0.j, ...,   0.+0.j,   0.+0.j,
             0.+0.j],
          [  0.+0.j, 730.+0.j,   0.+0.j, ...,   0.+0.j,   0.+0.j,
             0.+0.j],
          [  0.+0.j,   0.+0.j, 730.+0.j, ...,   0.+0.j,   0.+0.j,
             0.+0.j],
          ...,
          [  0.+0.j,   0.+0.j,   0.+0.j, ..., 734.+0.j,   0.+0.j,
             0.+0.j],
          [  0.+0.j,   0.+0.j,   0.+0.j, ...,   0.+0.j, 734.+0.j,
             0.+0.j],
          [  0.+0.j,   0.+0.j,   0.+0.j, ...,   0.+0.j,   0.+0.j,
           735.+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2))


In [129]:
result = mes.compute_minimum_eigenvalue(H0_qop)
print(result.eigenvalue)

(729+0j)
